In [6]:
# pip install azureml-sdk

In [7]:
## STEP 1: Set up the workspace.

import json

# GETTING MY ID:
id = open('my_id.json', 'r')
mi = json.load(id)
my_id = mi["my_id"]

# Crear el workspace
from azureml.core import Workspace

ws = Workspace.create(name = "Sales-LT",
                      subscription_id = my_id,
                      resource_group = "class4",
                      location = "centralindia")


# Mi modleo
from azureml.core.model import Model

mname = "modelo_modifieddate"

registered_model = Model.register(model_path="modelo_modifieddate.pkl",
                                  model_name=mname,
                                  workspace=ws)

Cleaning up past default Resource Group Deployments on the subscription to avoid limit of 10
Deleting past Resource Group Deployment with name: DeployResourceGroup-5f91fc1213
Deploying StorageAccount with name salesltstorage522f022341.
Deploying KeyVault with name salesltkeyvault490967b03.
Deploying AppInsights with name salesltinsights4672a022a.
Deployed AppInsights with name salesltinsights4672a022a. Took 54.35 seconds.
Deploying Workspace with name Sales-LT.
Deployed Workspace with name Sales-LT. Took 44.71 seconds.
Registering model modelo_modifieddate


In [8]:
scorepy = f"""
import json
import pickle
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path('{mname}')
    with open(model_path, 'rb') as file:
        model = pickle.load(file)

def run(raw_data):
    try:
        # Leer los datos recibidos
        data = json.loads(raw_data)['data']
        data = pd.DataFrame(data)

        # Predecir los días desde la fecha base
        pred_days = model.predict(data).tolist()

        fecha_base = pd.to_datetime(datos["ModifiedDate"].min())
        pred_dates = (fecha_base + pd.to_timedelta(pred_days, unit='D')).astype(str).tolist()

        return json.dumps(pred_dates)

    except Exception as e:
        return json.dumps({{"error": str(e)}})
"""

# Guardar el script como score.py
with open("score.py", "w") as file_score:
    file_score.write(scorepy)

In [9]:
## STEP 2: Inference configuration adaptado a tu modelo de regresión de fechas ---

from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

virtual_env = Environment("env-4-modifieddate")

virtual_env.python.conda_dependencies = CondaDependencies.create(
    conda_packages=['pandas', 'scikit-learn'],
    pip_packages=['azureml-defaults']
)

inference_config = InferenceConfig(
    environment=virtual_env,
    entry_script="score.py"
)

aci_config = AciWebservice.deploy_configuration(
    cpu_cores=0.5,
    memory_gb=1
)

service = Model.deploy(
    workspace=ws,
    name='modifieddate-service',
    models=[registered_model],
    inference_config=inference_config,
    deployment_config=aci_config,
    overwrite=True
)

C:\Users\danyj\AppData\Local\Temp\ipykernel_29580\3989220218.py:25: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


In [11]:
service.wait_for_deployment()

scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()

ACI service creation operation finished, operation "Succeeded"
